- https://github.com/tensorflow/models/tree/master/research/im2txt
- https://yashk2810.github.io/
- https://github.com/yashk2810/Image-Captioning/blob/master/Image%20Captioning%20InceptionV3.ipynb
- https://arxiv.org/pdf/1609.06647.pdf

In [4]:
from PIL import Image
from glob import glob
import os
import re
import pdb
import pickle

In [1]:
#dpath = '/data/image_captioning/flikr8/'
dpath = 'flikr8/'

img_path = dpath+'Flicker8k_Dataset/'


- 8091 images
- 5 captions for each image.
- maybe just take the first caption to start
- 6000 train images, 1000 dev images, 1000 test images
- will need to separate into different directories.
- will probably want dataset captions in different files.
- all images seem to be 500 tall
- https://yashk2810.github.io/
- https://github.com/yashk2810/Image-Captioning/blob/master/Image%20Captioning%20InceptionV3.ipynb
- now you have captions. You'll need to tokenize them and embed them.
- the next step would probably be to get the activations for the images
- if you want to precompute them you'll need to save them in some type of specified order so you can match them with the captions.
- 

In [3]:
imgs = glob(img_path+'/*.jpg'); len(imgs)

91

## Caption Processing

First, maybe just take the first caption for each image.
- construct a regex that matches the first captions

### Create file of only first captions

In [5]:
match = '1003163366_44323f5815.jpg#0 '
match_2 = '1003163366_44323f5815.jpg#2 '

In [6]:
first_cap_re = re.compile('^.+\.jpg#\d')

In [7]:
first_cap_re.match(match), first_cap_re.match(match_2)

(<_sre.SRE_Match object; span=(0, 27), match='1003163366_44323f5815.jpg#0'>,
 <_sre.SRE_Match object; span=(0, 27), match='1003163366_44323f5815.jpg#2'>)

In [8]:
all_caps_path = dpath+'Flickr8k.token.txt'
all_caps_dict = {}
for line in open(all_caps_path):
    if first_cap_re.match(line):
        line_arr = line.split('\t')
        img_nm = line_arr[0][:-2]
        cap = line_arr[1].strip()
        if all_caps_dict.get(img_nm, False):
            all_caps_dict[img_nm].append(cap)
        else:
            all_caps_dict[img_nm] = [cap]

FileNotFoundError: [Errno 2] No such file or directory: 'flikr8/Flickr8k.token.txt'

In [36]:
names = list(all_caps_dict.keys())
all_caps_dict[names[100]]

['A black and white Border Collie catches a Frisbee in front of an audience .',
 'a brown and white dog catches a Frisbee in it mouth in front of a group of people .',
 'A brown and white dug jumping up to catch a Frisbee while an audience watches .',
 'A dog jumps to catch a Frisbee , while many people watch .',
 'An agile dog catches a Frisbee while a crowd of onlookers watches closely .']

In [35]:
values = list(all_caps_dict.values()); 
lens =[len(v) for v in values]
min(lens), max(lens)

(5, 5)

In [38]:
pickle.dump(all_caps_dict, open(dpath+'all_caps.pkl', 'wb'))

In [40]:
all_caps = pickle.load(open(dpath+'all_caps.pkl', 'rb'));

### Separate train, dev and test captions

In [41]:
def getImgCapForSet(set_names_path, all_imgs_caps):    
    nms = [line.strip() for line in open(set_names_path)]
    return {k:v for k,v in all_imgs_caps.items() if k in nms} 


In [42]:
tr_nms_caps = getImgCapForSet(dpath+'Flickr_8k.trainImages.txt', all_caps)

In [50]:
nms = list(tr_nms_caps.keys())
len(tr_nms_caps), tr_nms_caps[nms[100]]

(6000,
 ['Two men are sitting on the ground and going through their backpacks .',
  'Two men sitting in front of a railing are looking at something .',
  'Two men with backpacks placed in front of them sitting in front of a railing .',
  'Two men with backpacks are sitting on cardboard along a railing .',
  'Two older men with backpacks sit at a bus station .'])

In [51]:
test_nms_caps = getImgCapForSet(dpath+'Flickr_8k.testImages.txt', all_caps)
nms = list(test_nms_caps.keys())
len(test_nms_caps), test_nms_caps[nms[100]]

(1000,
 ['A man and a girl sit on the ground and eat .',
  'A man and a little girl are sitting on a sidewalk near a blue bag eating .',
  'A man and young girl eat a meal on a city street .',
  'A man wearing a black shirt and a girl wearing an orange shirt sitting on the pavement eating .',
  'A man wearing a black shirt and a little girl wearing an orange dress share a treat .'])

In [52]:
dev_nms_caps = getImgCapForSet(dpath+'Flickr_8k.devImages.txt', all_caps)
nms = list(dev_nms_caps.keys())
len(dev_nms_caps), dev_nms_caps[nms[100]]

(1000,
 ['a couple with a young child wrapped in a blanket sitting on a concrete step',
  'A woman and a man holding a child sit on steps .',
  'A young family sits on steps enjoying the day',
  'A young family sitting on stone steps .',
  'The man and woman are sitting on wooden steps holding a baby .'])

In [54]:
pickle.dump(tr_nms_caps, open(dpath+'train_all_caps.pkl', 'wb'))
pickle.dump(test_nms_caps, open(dpath+'test_all_caps.pkl', 'wb'))
pickle.dump(dev_nms_caps, open(dpath+'dev_all_caps.pkl', 'wb'))

### Move Images

In [5]:
tr_img_names = [nm.strip() for nm in open(dpath+ 'Flickr_8k.trainImages.txt')]

tr_img_path = img_path+'train'
for nm in tr_img_names:
    os.rename(os.path.join(img_path, nm), os.path.join(tr_img_path, nm))

In [20]:
def moveImgs(nm_path, out_path, img_path=img_path):
    img_names = [nm.strip() for nm in open(nm_path)]
    for nm in img_names:
        os.rename(os.path.join(img_path, nm), os.path.join(out_path, nm))

In [21]:
moveImgs(dpath+ 'Flickr_8k.devImages.txt', img_path+'dev')

In [24]:
moveImgs(dpath+ 'Flickr_8k.testImages.txt', img_path+'test')

### Create Sample

In [6]:
from shutil import copy

In [5]:
samp_path =img_path+'samp/'

In [7]:
tr_imgs = glob(img_path+'train/wrap/*.jpg')

In [8]:
first32 = tr_imgs[:32]

In [13]:
next32 = tr_imgs[32:65]

In [10]:
len(first32)

32

In [20]:
def copyToPath(fs, p):
    for f in fs:
        copy(f, os.path.join(p, os.path.basename(f)))

In [21]:
copyToPath(first32, samp_path+'train/wrap/')
copyToPath(next32, samp_path+'val/wrap/')
